# SVM con validación cruzada K-Fold

## Importación de Librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Obteniendo los datos

In [2]:
mammographic_masses = pd.read_csv('./processed/mammographic_masses_clean.csv')

In [43]:
mammographic_masses.head(10)

,Age,Shape,Margin,Density,Severity
0,67,3,5,3,1
1,58,4,5,3,1
2,28,1,1,3,0
3,57,1,5,3,1
4,76,1,4,3,1
5,42,2,1,3,1
6,36,3,1,2,0
7,60,2,1,2,0
8,54,1,1,3,0
9,52,3,4,3,0


In [4]:
mammographic_masses.describe()

,Age,Shape,Margin,Density,Severity
count,831.000000,831.000000,831.000000,831.000000,831.000000
mean,55.777377,2.783394,2.814681,2.915764,0.484958
std,14.663528,1.242331,1.566771,0.350737,0.500075
min,18.000000,1.000000,1.000000,1.000000,0.000000
25%,46.000000,2.000000,1.000000,3.000000,0.000000
50%,57.000000,3.000000,3.000000,3.000000,0.000000
75%,66.000000,4.000000,4.000000,3.000000,1.000000
max,96.000000,4.000000,5.000000,4.000000,1.000000


In [5]:
mammographic_masses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831 entries, 0 to 830
Data columns (total 5 columns):
Age         831 non-null int64
Shape       831 non-null int64
Margin      831 non-null int64
Density     831 non-null int64
Severity    831 non-null int64
dtypes: int64(5)
memory usage: 32.5 KB


## Construyendo las matrices

In [34]:
X = mammographic_masses.iloc[:,0:4].values
y = mammographic_masses.iloc[:,4].values
X

array([[67,  3,  5,  3],
       [58,  4,  5,  3],
       [28,  1,  1,  3],
       ...,
       [64,  4,  5,  3],
       [66,  4,  5,  3],
       [62,  3,  3,  3]], dtype=int64)

## División en conjunto de entrenamiento y prueba

In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

## Entrenar el clasificador con Vectores de Soporte

In [88]:
from sklearn.svm import SVC

In [90]:
model = SVC()

In [91]:
model.fit(X_train,y_train)

C:\Users\CHRISTIAN\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

## Predicciones y evaluaciones

Ahora vamos a predecir haciendo uso del modelo entrenado.

In [92]:
predictions = model.predict(X_test)

In [93]:
from sklearn.metrics import classification_report,confusion_matrix

In [94]:
print(confusion_matrix(y_test,predictions))

[[71 35]
 [12 90]]


In [95]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.86      0.67      0.75       106
           1       0.72      0.88      0.79       102

   micro avg       0.77      0.77      0.77       208
   macro avg       0.79      0.78      0.77       208
weighted avg       0.79      0.77      0.77       208



## Validación Cruzada KFold

La solución a este problema es usar la validación cruzada K-Fold para la evaluación del desempeño, donde K es cualquier número. El proceso de validación cruzada K-Fold es sencillo. Se dividen los datos en K pliegues. De los pliegues K, los conjuntos K-1 se usan para entrenamiento mientras que el resto se usa para pruebas. El algoritmo se entrena y se prueba K veces, cada vez que se utiliza un nuevo conjunto como conjunto de prueba, mientras que los conjuntos restantes se utilizan para el entrenamiento. Finalmente, el resultado de la validación cruzada K-Fold es el promedio de los resultados obtenidos en cada conjunto.

In [96]:
from sklearn.model_selection import cross_val_score  
all_accuracies = cross_val_score(estimator=model, X=X_train, y=y_train, cv=10)  

C:\Users\CHRISTIAN\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\CHRISTIAN\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\CHRISTIAN\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\CHRISTIAN\Anaconda3\lib\site-packages\sklearn\svm\base.py:

A continuación mostramos las precisiones de cada iteración.

In [97]:
print(all_accuracies)

[0.75       0.79365079 0.77419355 0.62903226 0.83870968 0.88709677
 0.83870968 0.83870968 0.72580645 0.87096774]


Mostramos el promedio de todas las precisiones.

In [98]:
print(all_accuracies.mean())  

0.7946876600102406


Finalmente, encontremos la desviación estándar de los datos para ver el grado de variación en los resultados obtenidos por nuestro modelo. Para hacerlo, llame al método std () en el objeto all_accuracies.

In [99]:
print(all_accuracies.std())  

0.07401102151524971
